In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('ressources/time_series_covid19_confirmed_global_iso3_regions.csv')

In [4]:
data.drop(index=0, inplace=True)

In [5]:
data.set_index(['Country/Region','Region Name'], inplace=True)

In [6]:
data.columns[:3]

Index(['Province/State', 'Lat', 'Long'], dtype='object')

In [7]:
data.columns[-6:]

Index(['ISO 3166-1 Alpha 3-Codes', 'Region Code', 'Sub-region Code',
       'Sub-region Name', 'Intermediate Region Code',
       'Intermediate Region Name'],
      dtype='object')

In [8]:
covid19_confirmed = data.drop(columns=data.columns[-6:]).drop(columns=data.columns[:3]).reset_index().melt(id_vars=['Country/Region','Region Name'],var_name='Date', value_name='Cases')

In [9]:
covid19_newcases = data.drop(columns=data.columns[-6:]).drop(columns=data.columns[:3]).T.diff().fillna(0).T.reset_index().melt(id_vars=['Country/Region','Region Name'],var_name='Date', value_name='New Cases')

In [10]:
covid19 = pd.concat([covid19_newcases, covid19_confirmed['Cases']],axis=1)

In [11]:
covid19.Date = pd.to_datetime(covid19.Date, format='%m/%d/%y')

In [12]:
cumsum_cases = covid19[['Date','New Cases']].groupby('Date').sum().cumsum().reset_index()

In [13]:
covid19 = covid19.merge(cumsum_cases, how='left', right_on="Date", left_on="Date").rename(columns={'New Cases_x':'New Cases','Cases':'Case by country', 'New Cases_y':'Cases Global'})

In [14]:
covid19 = covid19.merge(covid19[['Date','New Cases']].groupby('Date').sum().reset_index(), how='left', right_on="Date", left_on="Date")

In [15]:
covid19.rename(columns={'New Cases_x':'New Cases','New Cases_y':'New Cases by day'}, inplace=True)

In [16]:
covid19.to_csv('covid19_Looker.csv', index=False)